In [10]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# DistrilBert 모델 Load


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig
from safetensors.torch import load_file
import os
import torch

# 모델 구성 로드
model_save_path = '/content/drive/MyDrive/bigdata_security/final_model_save_200_tuned'

# Config 파일을 저장된 경로에서 로드
config_path = os.path.join(model_save_path, 'config.json')
config = DistilBertConfig.from_pretrained(config_path)

# 모델 생성
model_v = DistilBertForSequenceClassification(config)
state_dict = load_file(os.path.join(model_save_path, 'model.safetensors'))

# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 가중치 로드
model_v.load_state_dict(state_dict)
model_v.to(device)  # GPU 사용 시
print("Model loaded successfully")

Model loaded successfully


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
import numpy as np

import torch

# 파일에서 텐서 로드
attention_masks = torch.load('/content/drive/MyDrive/bigdata_security/attendtion_masks.pt')
input_ids = torch.load('/content/drive/MyDrive/bigdata_security/input_ids.pt')

In [ ]:
predict_val = []
import pandas as pd
# 새로운 데이터프레임 생성
# ditrilbert = pd.DataFrame(columns = ['id','proba'])

ditrilbert = pd.DataFrame(columns = ['id', '0', '1', '2', '3', '4', '5', '6'])
ditrilbert = pd.read_csv('/content/drive/MyDrive/bigdata_security/model_submission/ensemble_ditrilbert.csv')
start_id = 1000000
print(len(ditrilbert))
model_v.config.max_position_embeddings, len(input_ids)

899900


(200, 1418916)

### Distribert 사용으로 인한 메모리 부족으로 100개 단위로 batchsize를 설정하여 예측 수행하여 제출용 파일 생성
140만개 데이터 예측에 2시간 이상이 소요되어 random으로 1/100 확률로 중간중간 저장하도록 구현


In [ ]:
import torch
import numpy as np
import pandas as pd
import time
import torch.nn.functional as F
import random

# Ensure that all tensors and the model are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_v.to(device)
input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)
batch_size = 100

model_v.eval()
for i in range(len(ditrilbert), len(input_ids), batch_size):
    start_id = 1000000+i
    if random.randint(0,100) == 0:
      print(f'{i}_th data is evaluating')
      ditrilbert.to_csv('/content/drive/MyDrive/bigdata_security/model_submission/ensemble_ditrilbert.csv', index=False)

    with torch.no_grad():
        outputs = model_v(input_ids[i:i+batch_size], attention_mask=attention_masks[i:i+batch_size])
        logits = outputs.logits

    # Softmax 함수를 Numpy로 대체하여 확률 계산
    exp_logits = F.softmax(logits, dim=-1)
    probas = exp_logits.cpu().numpy()
    # Numpy로 argmax 대체
    # max_indices = np.argmax(probas, axis=1)
    # max_probs = np.max(exp_logits.cpu().numpy(), axis=1)

    # predictions = np.where(max_probs < 0.66, 7, max_indices)
    prediction_id = np.arange(start_id, start_id +  len(probas))

    predictions_df = pd.DataFrame({'id': prediction_id,'0': probas[:,0],'1': probas[:,1],'2': probas[:,2],'3': probas[:,3],'4': probas[:,4],'5': probas[:,5],'6': probas[:,6]})

    ditrilbert = pd.concat([ditrilbert, predictions_df], ignore_index=True)
ditrilbert.to_csv('/content/drive/MyDrive/bigdata_security/model_submission/ensemble_ditrilbert.csv', index=False)
ditrilbert


899900_th data is evaluating
906400_th data is evaluating
942400_th data is evaluating
963900_th data is evaluating
993000_th data is evaluating
993300_th data is evaluating
1017000_th data is evaluating
1023000_th data is evaluating
1040200_th data is evaluating
1051000_th data is evaluating
1068600_th data is evaluating
1070300_th data is evaluating
1099700_th data is evaluating
1101500_th data is evaluating
1104500_th data is evaluating
1108700_th data is evaluating
1112900_th data is evaluating
1120900_th data is evaluating
1121900_th data is evaluating
1154400_th data is evaluating
1157800_th data is evaluating
1159200_th data is evaluating
1161000_th data is evaluating
1180100_th data is evaluating
1195700_th data is evaluating
1265700_th data is evaluating
1272000_th data is evaluating
1276500_th data is evaluating
1299200_th data is evaluating
1300900_th data is evaluating
1306600_th data is evaluating
1334600_th data is evaluating
1339400_th data is evaluating
1339900_th data 

,id,proba,0,1,2,3,4,5,6
0,1000000,NaN,0.983753,0.007042,0.000180,0.001563,0.000180,0.007066,0.000216
1,1000001,NaN,0.966352,0.003610,0.000480,0.002329,0.000224,0.026676,0.000330
2,1000002,NaN,0.011273,0.985264,0.000050,0.001940,0.000199,0.001134,0.000140
3,1000003,NaN,0.966352,0.003610,0.000480,0.002329,0.000224,0.026676,0.000330
4,1000004,NaN,0.009761,0.986834,0.000049,0.001855,0.000191,0.001172,0.000137
...,...,...,...,...,...,...,...,...,...
1418911,2418911,NaN,0.966352,0.003610,0.000480,0.002329,0.000224,0.026676,0.000330
1418912,2418912,NaN,0.968071,0.010105,0.000404,0.001283,0.000231,0.019535,0.000371
1418913,2418913,NaN,0.008808,0.987625,0.000053,0.001954,0.000208,0.001201,0.000149
1418914,2418914,NaN,0.981440,0.004606,0.000368,0.001586,0.000237,0.011456,0.000306


# GRU 모델 Load


In [1]:
import torch.nn as nn

class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(GRUClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=6, padding=3)
        self.conv4 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=8, padding=4)
        self.conv5 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=10, padding=5)
        self.conv6 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=12, padding=6)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.gru = nn.GRU(input_size=512*6, hidden_size=hidden_dim, batch_first=True, bidirectional=True, num_layers=1, dropout=0.3)
        self.batchnorm = nn.BatchNorm1d(512*6)
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)
        conv_outputs = []
        for conv in [self.conv1, self.conv2, self.conv3, self.conv4, self.conv5, self.conv6]:
            conv_output = conv(x)
            conv_output = F.elu(conv_output)
            conv_output = self.maxpool(conv_output)
            conv_output = self.avgpool(conv_output)
            conv_outputs.append(conv_output.squeeze(2))
        x = torch.cat(conv_outputs, dim=1)
        x = self.batchnorm(x)
        x = self.dropout(x)
        x, _ = self.gru(x.unsqueeze(1))
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [2]:
import torch

# 파일에서 모델 읽기
gru_model = torch.load('best_model_GRU_epoch10.pt', map_location=torch.device('cpu'))
gru_model

GRUClassifier(
  (embedding): Embedding(31014, 1024, padding_idx=0)
  (conv1): Conv1d(1024, 512, kernel_size=(2,), stride=(1,), padding=(1,))
  (conv2): Conv1d(1024, 512, kernel_size=(4,), stride=(1,), padding=(2,))
  (conv3): Conv1d(1024, 512, kernel_size=(6,), stride=(1,), padding=(3,))
  (conv4): Conv1d(1024, 512, kernel_size=(8,), stride=(1,), padding=(4,))
  (conv5): Conv1d(1024, 512, kernel_size=(10,), stride=(1,), padding=(5,))
  (conv6): Conv1d(1024, 512, kernel_size=(12,), stride=(1,), padding=(6,))
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AdaptiveAvgPool1d(output_size=1)
  (gru): GRU(3072, 512, batch_first=True, dropout=0.3, bidirectional=True)
  (batchnorm): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=1024, out_features=8, bias=True)
)

In [3]:
import pandas as pd
test = pd.read_csv('test.csv')
test

,id,full_log
0,1000000,"Feb 8 15:47:26 localhost kibana: {""type"":""err..."
1,1000001,"Sep 24 03:46:39 localhost kibana: {""type"":""err..."
2,1000002,type=SYSCALL msg=audit(1611888200.428:210563):...
3,1000003,"Jan 18 11:24:06 localhost kibana: {""type"":""err..."
4,1000004,type=SYSCALL msg=audit(1603081202.050:46851): ...
...,...,...
1418911,2418911,"Jan 13 05:07:11 localhost kibana: {""type"":""err..."
1418912,2418912,"Jan 5 02:24:50 localhost kibana: {""type"":""log..."
1418913,2418913,type=SYSCALL msg=audit(1611884593.462:38222): ...
1418914,2418914,Jan 22 01:38:19 localhost logstash: [2021-01-2...


In [4]:
import re
def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)
    # IP 주소 제거
    text = re.sub(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d{1,5}\b", 'IP', text)
    text = re.sub(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", 'IP', text)
    # 영어와 한글 외 문자 제거
    text = re.sub(r'[^a-zA-Z가-힣\s]', '', text)
    return text

In [5]:
test['full_log'] = test['full_log'].apply(preprocess_text) # 전처리 적용
test_df = test
test_df

,id,full_log
0,1000000,feb localhost kibana typeerrortimestamptzta...
1,1000001,sep localhost kibana typeerrortimestamptztag...
2,1000002,typesyscall msgaudit archce syscall successyes...
3,1000003,jan localhost kibana typeerrortimestamptztag...
4,1000004,typesyscall msgaudit archce syscall successyes...
...,...,...
1418911,2418911,jan localhost kibana typeerrortimestamptztag...
1418912,2418912,jan localhost kibana typelogtimestamptztags...
1418913,2418913,typesyscall msgaudit archce syscall successyes...
1418914,2418914,jan localhost logstash twarn logstashoutputs...


In [11]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 텍스트 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_df['full_log'])
sequences = tokenizer.texts_to_sequences(test_df['full_log'])

# 정수 시퀀스 패딩
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

input_data = torch.tensor(padded_sequences, dtype=torch.long).to(device)

In [57]:
import pandas as pd
gru = pd.DataFrame(columns = ['id', '0', '1', '2', '3', '4', '5', '6'])

In [58]:
import torch
import numpy as np
import torch.nn.functional as F
import random

batch_size = 100

for i in range(len(gru), len(input_data), batch_size):
    start_id = 1000000+i
    if random.randint(0,100) == 0:
        gru.to_csv("gru_probability.csv")
        print(i)

    output = gru_model(input_data[i:i+batch_size])
    
    probas = F.softmax(output, dim=1)
    probas = probas.detach().cpu().numpy()

    # 클래스별 확률을 numpy array로 변환
    prediction_id = np.arange(start_id, start_id +  len(probas))
    predictions_df = pd.DataFrame({'id': prediction_id,'0': probas[:,0],'1': probas[:,1],'2': probas[:,2],'3': probas[:,3],'4': probas[:,4],'5': probas[:,5],'6': probas[:,6]})

    gru = pd.concat([gru, predictions_df], ignore_index=True)
gru.to_csv("gru_probability.csv")

KeyboardInterrupt: 

# RF 모델 Load

In [ ]:
import torch
import joblib

test =torch.load('/content/drive/MyDrive/bigdata_security/rf_test_features.pt')
test

<1418916x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 21255422 stored elements in Compressed Sparse Row format>

In [ ]:
model = joblib.load('/content/drive/MyDrive/bigdata_security/best_rf_model.joblib')
# results = model.predict(test)
results_proba = model.predict_proba(test)
results_proba

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import pandas as pd
probas = results_proba
rf = pd.DataFrame({'id': range(1000000,2418916),'0': probas[:,0],'1': probas[:,1],'2': probas[:,2],'3': probas[:,3],'4': probas[:,4],'5': probas[:,5],'6': probas[:,6]})
rf

In [ ]:
rf.to_csv("/content/drive/MyDrive/bigdata_security/rf_proba.csv",index = False)

# Soft Voting csv기준으로

In [11]:
import pandas as pd
rf = pd.read_csv("rf_proba.csv")
distrilbert = pd.read_csv("ensemble_ditrilbert.csv")
gru = pd.read_csv("gru_proba.csv")
len(rf), len(distrilbert) , len(gru)

(1418916, 1418916, 1418916)

In [14]:
soft_voting = distrilbert + rf + gru
soft_voting['id'] = soft_voting['id']//3
soft_voting

,id,0,1,2,3,4,5,6
0,1000000,2.972388,0.007302,0.000245,0.011064,0.000588,0.007074,0.001028
1,1000001,2.966228,0.003610,0.000482,0.002425,0.000235,0.026676,0.000339
2,1000002,0.011280,2.984951,0.000056,0.001959,0.000210,0.001382,0.000147
3,1000003,2.966163,0.003610,0.000484,0.002480,0.000239,0.026676,0.000342
4,1000004,0.011324,2.965776,0.000103,0.003233,0.000631,0.017975,0.000247
...,...,...,...,...,...,...,...,...
1418911,2418911,2.966163,0.003610,0.000484,0.002480,0.000239,0.026676,0.000342
1418912,2418912,2.968033,0.010115,0.000404,0.001293,0.000233,0.019537,0.000384
1418913,2418913,0.008813,2.987200,0.000059,0.001968,0.000232,0.001556,0.000157
1418914,2418914,2.981413,0.004606,0.000369,0.001605,0.000238,0.011457,0.000311


In [47]:
import numpy as np
probabilities = soft_voting[['0','1','2','3','4','5','6']].to_numpy()
max_probs = np.max(probabilities,axis = 1)/3
max_indices = np.argmax(probabilities,axis=1)

predictions = np.where(max_probs < 0.55, 7, max_indices)

submissoin_ensemble = pd.DataFrame({"id" : soft_voting['id'], 'level' : predictions})
submissoin_ensemble.to_csv("submission_ensemble.csv",index = False)
submissoin_ensemble

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [48]:
submissoin_ensemble['level'].value_counts()

0    1002907
1     395700
3      12996
5       6392
7        825
2         34
4         34
6         28
Name: level, dtype: int64

In [35]:
rf
import numpy as np
probabilities = rf[['0','1','2','3','4','5','6']].to_numpy()
max_probs = np.max(probabilities,axis = 1)/3
max_indices = np.argmax(probabilities,axis=1)

predictions = np.where(max_probs < 0.8, 7, max_indices)

submissoin_ensemble = pd.DataFrame({"id" : soft_voting['id'], 'level' : predictions})
submissoin_ensemble.to_csv("submission_rf.csv",index = False)
submissoin_ensemble

,id,level
0,1000000,7
1,1000001,7
2,1000002,7
3,1000003,7
4,1000004,7
...,...,...
1418911,2418911,7
1418912,2418912,7
1418913,2418913,7
1418914,2418914,7


# Softvoting 코드 기준

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig
from safetensors.torch import load_file
import os
import torch
# 모델 load
# RF
rf_model = joblib.load('/content/drive/MyDrive/bigdata_security/best_rf_model.joblib')

# gru
gru_model = torch.load('/content/drive/MyDrive/bigdata_security/best_model_GRU_epoch10.pt', map_location=torch.device('cpu'))

# DistrilBert
model_save_path = '/content/drive/MyDrive/bigdata_security/final_model_save_200_tuned'

# Config 파일을 저장된 경로에서 로드
config_path = os.path.join(model_save_path, 'config.json')
config = DistilBertConfig.from_pretrained(config_path)

# 모델 생성
distrilbert_model = DistilBertForSequenceClassification(config)
state_dict = load_file(os.path.join(model_save_path, 'model.safetensors'))

# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 가중치 로드
distrilbert_model.load_state_dict(state_dict)
distrilbert_model.to(device)  # GPU 사용 시
print("Model loaded successfully")

In [ ]:
import torch
import numpy as np
import pandas as pd
import time
import torch.nn.functional as F

# Ensure that all tensors and the model are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_v.to(device)
input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)

batch_size = 100
submission_ensemble = pd.DataFrame(columns = ['id', '0', '1', '2', '3', '4', '5', '6'])

distrilbert_model.eval()
for i in range(len(submission_ensemble), len(input_ids), batch_size):
    start_id = 1000000+i
    if random.randint(0,100) == 0:
      print(f'{i}_th data is evaluating')
      submission_ensemble.to_csv('/content/drive/MyDrive/bigdata_security/model_submission/ensemble_submission.csv', index=False)

    input_batch = input_ids[i:i+batch_size]
    # Bert Predict
    with torch.no_grad():
        outputs = distrilbert_model(input_batch, attention_mask=attention_masks[i:i+batch_size])
        logits = outputs.logits
    exp_logits = F.softmax(logits, dim=-1)
    bert_probas = exp_logits.cpu().numpy()

    # RF Predict
    rf_probas = model.predict_proba(input_batch)

    # GRU Predict
    output = gru_model(input_batch)
    probas = F.softmax(output, dim=1)
    gru_probas = probas.detach().cpu().numpy()
    
    # Soft voting
    total_probas = (gru_probas+rf_probas + bert_probas)/3
    max_indices = np.argmax(total_probas, axis=1)
    max_probs = np.max(total_probas, axis=1)

    # 7라벨 분류
    predictions = np.where(max_probs < 0.7, 7, max_indices)
    prediction_id = np.arange(start_id, start_id +  len(probas))

    # 제출용 dataframe에 추가
    predictions_df = pd.DataFrame({'id': prediction_id,'level': predictions})
    submission_ensemble = pd.concat([submission_ensemble, predictions_df], ignore_index=True)
submission_ensemble.to_csv('/content/drive/MyDrive/bigdata_security/model_submission/ensemble_submission.csv', index=False)
submission_ensemble

# 하드보팅

In [2]:
import pandas as pd
rf = pd.read_csv("rf_proba.csv")
distrilbert = pd.read_csv("ensemble_ditrilbert.csv")
gru = pd.read_csv("gru_proba.csv")
len(rf), len(distrilbert) , len(gru)

(1418916, 1418916, 1418916)

In [27]:
import numpy as np
from collections import Counter
df = gru[['0','1','2','3','4','5','6']].values
max_indices = np.argmax(df, axis=1)
max = np.max(df, axis=1)
predictions = np.where(max < 0.7, 7, max_indices)
counts = Counter(predictions)
print(counts)

for i,line in enumerate(max_indices):
    if line == 1:
        if max[i] > 0.5:
            predictions[i] = 1

predictions_gru = predictions
counts = Counter(predictions_gru)
print(counts)

Counter({0: 1001552, 1: 394204, 3: 12114, 5: 6362, 7: 4588, 2: 34, 4: 34, 6: 28})
Counter({0: 1001552, 1: 395626, 3: 12114, 5: 6362, 7: 3166, 2: 34, 4: 34, 6: 28})


In [29]:
import numpy as np

df = distrilbert[['0','1','2','3','4','5','6']].values
max_indices = np.argmax(df, axis=1)
max = np.max(df, axis=1)
predictions = np.where(max < 0.7, 7, max_indices)
predictions_distrilbert = predictions
counts = Counter(predictions_distrilbert)
counts

Counter({0: 1002101,
         1: 395215,
         3: 13053,
         7: 2039,
         5: 6412,
         2: 34,
         4: 34,
         6: 28})

In [36]:

import numpy as np
from collections import Counter

level_data = {
    0: 0.48,
    1: 0.47,
    2: 0.5,
    3: 0.45,
    4: 0.5,
    5: 0.41,
    6: 0.6
}

df = rf[['0','1','2','3','4','5','6']].values
max_indices = np.argmax(df, axis=1)
max = np.max(df, axis=1)
predictions = np.array([0]*len(max_indices))

for i,(index,max) in enumerate(zip(max_indices,max)):
    if max < level_data[index]:
        predictions[i] = 7
    else: predictions[i] = index
predictions_rf = predictions
counts = Counter(predictions_rf)
print(counts)

Counter({0: 1002598, 1: 396514, 3: 12976, 5: 6512, 7: 223, 2: 34, 4: 34, 6: 25})


In [47]:
# 모든 예측 배열을 합치기
preds = np.array([predictions_rf,predictions_distrilbert, predictions_gru])
global count
count = 0

# 각 인덱스마다 예측값을 모아서 투표 진행
def hard_voting(preds):
    global count
    final_preds = []
    for i in range(preds.shape[1]):
        # 해당 인덱스의 예측값들
        votes = preds[:, i]
        # 각 예측값의 빈도수 계산
        vote_counts = Counter(votes)
        # 가장 많이 나온 예측값 찾기
        most_common = vote_counts.most_common()
        
        if len(most_common) == 1 or most_common[0][1] > 1:
            # 예측값이 모두 같거나, 가장 많이 나온 예측값이 여러번 나온 경우
            final_preds.append(most_common[0][0])
        else:
            # 모든 예측값이 다 다른 경우, 첫번째 모델의 예측값을 사용
            final_preds.append('')
            count+=1
    
    return np.array(final_preds)

# 하드 보팅 결과
final_predictions = hard_voting(preds)
print(final_predictions)
count

['0' '0' '1' ... '1' '0' '0']


690

In [42]:

ids = np.array(range(1000000,1000000+len(final_predictions)))

submission_ensemble = pd.DataFrame({'id' : ids, 'level' : final_predictions})
submission_ensemble.to_csv("hard_voting.csv",index = False)
submission_ensemble

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


# 하드보팅 코드 기준

In [ ]:
# 각 인덱스마다 예측값을 모아서 투표 진행
def hard_voting(preds):
    final_preds = []
    for i in range(preds.shape[1]):
        votes = preds[:, i]
        # 각 예측값의 빈도수 계산
        vote_counts = Counter(votes)
        # 가장 많이 나온 예측값 찾기
        most_common = vote_counts.most_common()
        
        if len(most_common) == 1 or most_common[0][1] > 1:
            # 예측값이 모두 같거나, 가장 많이 나온 예측값이 여러번 나온 경우 정상수행
            final_preds.append(most_common[0][0])
        else:
            # 모든 예측값이 다 다른 경우, RF 모델 사용
            final_preds.append(preds[0, i])
    
    return np.array(final_preds)

In [ ]:
import torch
import numpy as np
import pandas as pd
import time
import torch.nn.functional as F

# Ensure that all tensors and the model are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_v.to(device)
input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)

batch_size = 100
submission_ensemble = pd.DataFrame(columns = ['id', 'level'])

distrilbert_model.eval()
for i in range(len(submission_ensemble), len(input_ids), batch_size):
    start_id = 1000000+i
    if random.randint(0,100) == 0:
      print(f'{i}_th data is evaluating')
      submission_ensemble.to_csv('/content/drive/MyDrive/bigdata_security/model_submission/hard_voting.csv', index=False)

    input_batch = input_ids[i:i+batch_size]
    # Bert Predict th 0.6설정
    with torch.no_grad():
        outputs = distrilbert_model(input_batch, attention_mask=attention_masks[i:i+batch_size])
        logits = outputs.logits
    exp_logits = F.softmax(logits, dim=-1)
    bert_probas = exp_logits.cpu().numpy()
    max_indices = np.argmax(bert_probas, axis=1)
    max = np.max(bert_probas, axis=1)
    predictions = np.where(max < 0.6, 7, max_indices)

    # RF Predict 각 th 다르게 설정
    rf_probas = model.predict_proba(input_batch)
    level_data = {0: 0.48, 1: 0.47, 2: 0.5, 3: 0.45, 4: 0.5, 5: 0.41, 6: 0.6}

    max_indices = np.argmax(rf_probas, axis=1)
    max = np.max(rf_probas, axis=1)
    predictions = np.array([0]*len(max_indices))

    for i,(index,max) in enumerate(zip(max_indices,max)):
        if max < level_data[index]:
            predictions[i] = 7
        else: predictions[i] = index
    predictions_rf = predictions

    # GRU Predict 1일때는 th 0.5 , 나머지는 0.7로 설정
    output = gru_model(input_batch)
    probas = F.softmax(output, dim=1)
    gru_probas = probas.detach().cpu().numpy()
    max_indices = np.argmax(gru_probas, axis=1)
    max = np.max(gru_probas, axis=1)
    predictions = np.where(max < 0.7, 7, max_indices)

    for i,line in enumerate(max_indices):
        if line == 1:
            if max[i] > 0.5:
                predictions[i] = 1

    predictions_gru = predictions
    
    # Hard Voting 수행, 단 모두 다른 라벨을 선택시 RF 기준으로 가도록 작성
    preds = np.array([predictions_rf,predictions_distrilbert, predictions_gru])

    final_predictions = hard_voting(preds)
    
    # 제출용 dataframe에 추가
    predictions_df = pd.DataFrame({'id': prediction_id,'level': predictions})
    submission_ensemble = pd.concat([submission_ensemble, predictions_df], ignore_index=True)
submission_ensemble.to_csv('/content/drive/MyDrive/bigdata_security/model_submission/ensemble_submission.csv', index=False)
submission_ensemble